<a href="https://colab.research.google.com/github/ramonVDAKKER/teaching-quantitative-finance/blob/main/notebooks/illustration_mc_methods_for_greeks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook illustration Monte Carlo approximation to the greeks

In this notebook we consider a simple situation: we use the standard Black-Scholes market and consider the verga of a European call option. Of course, we have a closed-form formula available for this vega. We will approximate the vega by the Monte Carlo method.


## 0. Imports

In [ ]:
%%capture
!git clone https://github.com/ramonVDAKKER/teaching-quantitative-finance
import os
os.chdir("teaching-quantitative-finance/notebooks")
import numpy as np
from scipy.stats import norm
from utils.black_scholes import BlackScholesOptionPrice

## 1. Example

Parameters:

In [ ]:
# B:
r = 0.01
# S:
S_0 = 100
sigma = 0.2
# call:
T = 1
K = 100

Vega, at $t=0$, using closed-form formula:

In [ ]:
call = BlackScholesOptionPrice(K, r, sigma)
vega = call.vega_call(current_stock_price=S_0, time_to_maturity=T)
print(f"Exact vega: {vega}")

### 1.1 Bump and Reprice using non-common random numbers (and one-sided finite difference)

In [ ]:
def approximate_vega_call_bump_reprice_osfd_noncommon(num_replications, T, sigma, r, K, h):

    def aux(T, r, sigma):
        S_T =  S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma *  np.sqrt(T) * norm.rvs(size=num_replications))
        option_price_prox = np.exp(-r * T) * np.mean(np.maximum(S_T - K, 0))
        return option_price_prox
    return (aux(T, r, sigma + h) - aux(T, r, sigma)) / h

In [ ]:
# Number of MC replications
R = 100000
estimate_vega_osfd_noncommon = approximate_vega_call_bump_reprice_osfd_noncommon(R, T, sigma, r, K, h=0.01)
print(f"Estimated vega on basis of Bump and Reprice with one-sided finite-difference and using noncommon random numbers: {estimate_vega_osfd_noncommon}")

### 1.2 Bump and Reprice using common random numbers (and one-sided finite difference)

In [ ]:
def approximate_vega_call_bump_reprice_osfd_common(num_replications, T, sigma, r, K, h):

    W_Q_T = np.sqrt(T) * norm.rvs(size=num_replications)
    S_T =  S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * W_Q_T)
    option_price_prox = np.exp(-r * T) * np.mean(np.maximum(S_T - K, 0))
    S_T_bump =  S_0 * np.exp((r - 0.5 * (sigma + h) ** 2) * T + (sigma + h) * W_Q_T)
    option_price_prox_bump = np.exp(-r * T) * np.mean(np.maximum(S_T_bump - K, 0))
    return (option_price_prox_bump - option_price_prox) / h

In [ ]:
estimate_vega_osfd_common = approximate_vega_call_bump_reprice_osfd_common(R, T, sigma, r, K, h=0.01)
print(f"Estimated vega on basis of Bump and Reprice with one-sided finite-difference and using common random numbers: {estimate_vega_osfd_common}")